In [3]:
import torch
from kan import *
import numpy as np

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_friedman2
import pickle

def get_dataset(n_samples, noise):
    seed = 42
    np.random.seed(seed)
    torch.manual_seed(seed)
    X_train, y_train = make_friedman2(n_samples=int(0.8 * n_samples), random_state =seed, noise = noise)
    X_test,y_test = make_friedman2(n_samples=int(0.2 * n_samples), noise = 0.)
    with open('fried_2_scaler_X.pkl', 'rb') as f:
        scaler_X = pickle.load(f)
    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)

    dataset_friedmann = {
        'train_input': torch.tensor(X_train, dtype = torch.float32),
        'test_input': torch.tensor(X_test, dtype = torch.float32),
        'train_label': torch.tensor(y_train, dtype = torch.float32).unsqueeze(1),
        'test_label': torch.tensor(y_test, dtype = torch.float32).unsqueeze(1)
    }
    return dataset_friedmann



In [8]:
import time
import dill
adjust = True 
widths = [10,20,50] ## Grids| train_loss: 1.92e+00 | test_loss: 8.15e-01 | reg: 0.00e+00 | :  44%|▍| 44/100 [37:20<47:16, 50.65s

n_samples = 20000
in_dims = [4]
noises = [2,5]
train_losses = []
test_losses = []
for in_dim in in_dims:
    for noise in noises:
        model = KAN(width=[in_dim,64,1], grid=10, k=3)
        dataset = get_dataset(n_samples = 20000, noise = noise)
        print(in_dim, noise)
        results = model.fit(dataset, opt="LBFGS", steps= 100, stop_grid_update_step=30, batch = -1)
        with open(f"models/Friedmann_2_retake_KAN_spline_arb_width_{noise}_{in_dim}.dill", "wb") as f:
            dill.dump(model, f)

checkpoint directory created: ./model
saving model version 0.0
4 2


| train_loss: 1.80e+00 | test_loss: 9.39e-01 | reg: 5.10e+02 | : 100%|█| 100/100 [1:27:02<00:00, 52.


saving model version 0.1
checkpoint directory created: ./model
saving model version 0.0
4 5


| train_loss: 4.22e+00 | test_loss: 2.83e+00 | reg: 5.05e+02 | : 100%|█| 100/100 [1:28:48<00:00, 53.


saving model version 0.1


In [ ]:
from sklearn.preprocessing import StandardScaler
def get_loader(scaler_X, noise, n_samples = 20000):
    seed = 42
    np.random.seed(seed)
    torch.manual_seed(seed)
    X_train, y = make_friedman2(n_samples= n_samples, random_state =seed, noise=noise)
    y_train = np.expand_dims(y, axis=1)
    X_train = scaler_X.transform(X_train)  

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

    # Create TensorDataset for train and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Set batch size and create DataLoader for training and testing
    batch_size = n_samples
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    return train_loader

@torch.no_grad()
def compute_test_loss(test_loader, model):
    criterion = torch.nn.MSELoss()
    running_loss = 0.
    for batch, target in test_loader:
        outputs = model(batch)
        loss = criterion(target, outputs)
        running_loss += loss.item()
    return running_loss / len(test_loader)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
from sklearn.datasets import make_friedman2
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import Kan_NN
import importlib
importlib.reload(Kan_NN)
import pickle
with open('fried_2_scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

In [ ]:
#### Evaluate models via for loop
import time
import dill
n_samples = 30000

test_loader = get_loader(scaler_X, noise = 0,n_samples =n_samples)
loss = compute_test_loss(test_loader, model)
print(loss)



            

0.20290407538414001
